# Sequence to sequence learning for performing number addition

### Setup

In [17]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


from tqdm import tqdm

In [16]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS


### Generate the data

In [2]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))


Generating data...
Total questions: 50000


### Print last 10 questions 

In [3]:
from pprint import pprint
pprint(questions[:10])

['   1+65',
 '  5+681',
 ' 07+454',
 '   0+15',
 '    1+6',
 ' 702+58',
 '  654+1',
 '  958+2',
 '  0+162',
 '    9+9']


### Vectorize the data

In [4]:
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype = bool)

y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype = bool)

In [13]:
print("dimension of x : ", x.shape)
print("dimension of y : ", y.shape)

dimension of x :  (50000, 7, 12)
dimension of y :  (50000, 4, 12)


### Encode the data

In [5]:
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

### Shuffle x and y

In [6]:
indices = np.arange(len(y))
np.random.shuffle(indices)

x = x[indices]
y = y[indices]

print("dimension of x : ", x.shape)
print("dimension of y : ", y.shape)

dimension of x :  (50000, 7, 12)
dimension of y :  (50000, 4, 12)


In [29]:
# Explicitly set apart 10% for validation data that we never train over.
# split_at = len(x) - len(x) // 10
# (x_train, x_test) = x[:split_at], x[split_at:]
# (y_train, y_test) = y[:split_at], y[split_at:]

### Split data in train and test

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
                    x,
                    y, 
                    test_size = 0.2, 
                    random_state = 42
)

In [8]:
print("dimension of x_train : ", x_train.shape)
print("dimension of y_train : ", y_train.shape)
print("dimension of x_test : ", x_test.shape)
print("dimension of y_test : ", y_test.shape)

dimension of x_train :  (40000, 7, 12)
dimension of y_train :  (40000, 4, 12)
dimension of x_test :  (10000, 7, 12)
dimension of y_test :  (10000, 4, 12)


### Create model

In [9]:
import tensorflow
num_layers = 1

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN can be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))

#previous numpy version numpy-1.22.3 generated errors like :
#NotImplementedError: Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

#so according to https://github.com/tensorflow/models/issues/9706#issuecomment-782841778
# dowgrade numpy to 1.19.5

2022-09-27 15:01:55.146522: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-27 15:01:55.147268: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-27 15:01:55.148132: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-27 15:01:55.148147: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nsl56): /proc/driver/nvidia/version does not exist
2022-09-27 15:01:55.148323: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flag

### Compile the model

In [10]:
model.compile(
    loss="categorical_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

### Model Summary

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [21]:

tensorflow.keras.utils.plot_model(model, 
                                  show_shapes=True, 
                                  show_layer_names=True, 
                                  expand_nested=True, 
                                  dpi=50, 
                                  to_file="perform_number_addition_using_seq_to_seq_learning_LSTM.png"
                        )



('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


### Train the model
Train the model on each epochs and predict the validation data

In [19]:


epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in tqdm(range(1, epochs)):
    print()
    print("\nIteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_test, y_test),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_test))
        rowx, rowy = x_test[np.array([ind])], y_test[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


  0%|                                                                                                                                                                               | 0/29 [00:00<?, ?it/s]2022-09-27 15:16:26.843978: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-27 15:16:26.862804: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2592000000 Hz




Iteration 1
1250/1250 [==============================] - 6s 4ms/step - loss: 1.8925 - accuracy: 0.3209 - val_loss: 1.5783 - val_accuracy: 0.4104


  3%|█████▊                                                                                                                                                                 | 1/29 [00:06<03:10,  6.82s/it]

Q 849+56  T 905  ☒ 900 
Q 93+66   T 159  ☒ 763 
Q 371+868 T 1239 ☒ 1222
Q 801+54  T 855  ☒ 900 
Q 710+448 T 1158 ☒ 1000
Q 65+85   T 150  ☒ 666 
Q 582+226 T 808  ☒ 100 
Q 302+18  T 320  ☒ 220 
Q 6+236   T 242  ☒ 226 
Q 48+57   T 105  ☒ 11  


Iteration 2
1250/1250 [==============================] - 5s 4ms/step - loss: 1.4175 - accuracy: 0.4709 - val_loss: 1.2287 - val_accuracy: 0.5469
Q 94+620  T 714  ☒ 700 
Q 525+82  T 607  ☒ 600 
Q 976+737 T 1713 ☒ 1742
Q 780+68  T 848  ☒ 855 
Q 7+403   T 410  ☒ 418 
Q 43+700  T 743  ☒ 745 
Q 910+199 T 1109 ☒ 1150


  7%|███████████▌                                                                                                                                                           | 2/29 [00:12<02:39,  5.89s/it]

...

Q 44+709  T 753  ☑ 753 
Q 37+77   T 114  ☑ 114 
Q 95+152  T 247  ☑ 247 
Q 863+5   T 868  ☑ 868 
Q 506+5   T 511  ☑ 511 
Q 931+19  T 950  ☑ 950 
Q 75+323  T 398  ☑ 398 
Q 29+63   T 92   ☑ 92  
Q 99+58   T 157  ☑ 157 
Q 665+35  T 700  ☑ 700 


Iteration 28
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0336 - accuracy: 0.9909 - val_loss: 0.0287 - val_accuracy: 0.9922


 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 28/29 [02:39<00:05,  5.84s/it]

Q 72+189  T 261  ☑ 261 
Q 8+19    T 27   ☑ 27  
Q 739+977 T 1716 ☑ 1716
Q 1+448   T 449  ☑ 449 
Q 61+666  T 727  ☑ 727 
Q 714+373 T 1087 ☒ 1077
Q 50+624  T 674  ☑ 674 
Q 378+133 T 511  ☑ 511 
Q 8+24    T 32   ☑ 32  
Q 380+62  T 442  ☑ 442 


Iteration 29
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0204 - accuracy: 0.9948 - val_loss: 0.0169 - val_accuracy: 0.9956


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [02:45<00:00,  5.71s/it]

Q 56+693  T 749  ☑ 749 
Q 36+711  T 747  ☑ 747 
Q 208+808 T 1016 ☑ 1016
Q 795+20  T 815  ☑ 815 
Q 833+769 T 1602 ☑ 1602
Q 9+406   T 415  ☑ 415 
Q 0+31    T 31   ☑ 31  
Q 70+308  T 378  ☑ 378 
Q 37+296  T 333  ☑ 333 
Q 77+427  T 504  ☑ 504 


In [18]:
# from tqdm import tqdm
# for i in tqdm(range(100000)):
#     print(i)

# Reference 

- https://keras.io/examples/nlp/addition_rnn/
- https://medium.com/@nutanbhogendrasharma/sequence-to-sequence-learning-with-neural-networks-to-perform-number-addition-7ef0df7afaa4